In [ ]:
import pandas as pd
import numpy as np

In [ ]:
all_news = pd.read_csv("processed_news_data.csv")

In [ ]:
from ast import literal_eval
all_news["tokens"] = all_news["tokens"].apply(literal_eval)

In [ ]:
all_news["bigrams"] = all_news["bigrams"].apply(literal_eval)

In [ ]:
def make_spacy_doc(row):
    print("we are at ",str(row["date"])
    return nlp(row["sentences"])

In [ ]:
all_news["sentence_en"] = all_news.apply(make_spacy_doc,axis=1)

In [ ]:
type(all_news["sentence_en"].iloc[0])

In [ ]:
def get_bigram(row):
    print("reaching "+str(row["date"]))
    a = []
    #print(len(row["tokens"]))
    for i in range(len(row["tokens"])-1):
        this_bigram = row["tokens"][i]+"_"+row["tokens"][i+1]
        a.append(this_bigram)
    
    return a

In [ ]:
backup_copy = all_news.copy(deep=True)

In [ ]:
get_bigram(all_news.iloc[0])

In [ ]:
all_news["bigrams"] = all_news.apply(get_bigram,axis=1)

In [ ]:
all_news["date"] = pd.to_datetime(all_news["date"], format="%Y-%m-%d")

In [ ]:
backup_all_news = all_news.copy(deep=True)
usable_all_news = all_news.copy(deep=True)

In [ ]:
length = len(all_news)

In [ ]:
train_news = all_news[0: int(length*4/5)]
test_news = all_news[int(length*4/5):length]

In [ ]:
processed_news = np.array(train_news)
sentences = processed_news[:,3]
tokens = processed_news[:,4]
sentence_handler = processed_news[:,5]
bigrams = processed_news[:,6]
company_symbol = processed_news[:,7]
company_name = processed_news[:,8]
company_processed_name = processed_news[:,9]
roc_future_past1 = 100*(processed_news[:,10] - processed_news[:,11])/processed_news[:,11]
roc_past1_past2 = 100*(processed_news[:,11] - processed_news[:,12])/processed_news[:,12]
roc_past2_past3 = 100*(processed_news[:,12] - processed_news[:,13])/processed_news[:,13]
price_movement=processed_news[:,14]



In [ ]:
from gensim.models import Word2Vec
model=Word2Vec(sentences=tokens,min_count=2)
bag_of_keywords=set(['rise','drop','fall','surge','shrink','jump','slump','reduce','outperform','progress','up','down'])
max_size=1000

for i in range(10):
    new_keywords=[]

    for word in bag_of_keywords:
        
        if word in model.wv.vocab.keys():
            
            new_keywords.extend(model.most_similar(word))
            print(len(new_keywords),model.most_similar(word))
            
    for new in new_keywords:
        if (new[0].isalpha()) and (new[0].islower()) and (len(new[0])>3):
            bag_of_keywords.add(new[0])
            
            if len(bag_of_keywords) == max_size:
                
                break

In [ ]:
bag_of_keywords=np.array(list(bag_of_keywords))
bag_of_keywords.shape

In [ ]:
from gensim.models import Word2Vec,phrases

from gensim.models.phrases import Phrases

all_bigrams = train_news["tokens"].iloc[:].values
bi = Phrases(all_bigrams)
model_bigram =Word2Vec(bi[all_bigrams],min_count=2)


In [ ]:
bag_of_bigram_keywords=set(['worse_than','percent_drop','stronger_than','economic_downturn',"further_damage","further_damage","economic_recover","help_boost","well_above","market_dominance"])

max_size_bigram = 200

for i in range(10):
    print(i)
    new_bigram_keywords=[]

    for word in bag_of_bigram_keywords:
        
        if word in model_bigram.wv.vocab.keys():
            
            new_bigram_keywords.extend(model_bigram.most_similar(word))
            print(len(new_bigram_keywords),model_bigram.most_similar(word))
            
    for new in new_bigram_keywords:
        
        if (new[0].isalpha()) and (new[0].islower()) and (len(new[0])>3):
            bag_of_bigram_keywords.add(new[0])
            
            if len(bag_of_bigram_keywords) == max_size_bigram:
                
                break
    if len(bag_of_bigram_keywords) == max_size_bigram:
        break

In [ ]:
bag_of_bigram_keywords =np.array(list(bag_of_bigram_keywords)) 
bag_of_bigram_keywords.shape

# Unigram Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(lowercase=False,min_df=1,use_idf=True,vocabulary=bag_of_keywords)


In [ ]:
news_tfidf = tfidf_vectorizer.fit_transform(sentences)
news_tfidf = news_tfidf.toarray()

In [ ]:
word_frequency = np.sum(news_tfidf,axis=0)

In [ ]:
positive_word_frequency = np.sum(news_tfidf[price_movement[:]==1.0],axis=0)*news_tfidf.shape[0]
positive_word_frequency = np.array(positive_word_frequency).reshape(news_tfidf.shape[1],)
positive_word_frequency[positive_word_frequency == 0] = 1
frequency_positive = np.sum(price_movement[:]==1.0)
PMI_positive = np.log(positive_word_frequency*sentences.shape[0]/\
                      word_frequency*frequency_positive)

In [ ]:
negative_word_frequency = np.sum(news_tfidf[price_movement[:]==-1.0],axis=0)*news_tfidf.shape[0]
negative_word_frequency = np.array(negative_word_frequency).reshape(news_tfidf.shape[1],)
negative_word_frequency[negative_word_frequency==0] = 1
frequency_negative = np.sum(price_movement[:]==-1.0)
PMI_negative = np.log(negative_word_frequency*sentences.shape[0]/\
                      word_frequency*frequency_negative)

In [ ]:
Polarity_Score = PMI_positive - PMI_negative
Polarity_Score.shape

In [ ]:
#Polarity_Score

In [ ]:
train_tfidf = TfidfVectorizer(lowercase=False,min_df=1,vocabulary=bag_of_keywords,use_idf=False)
X_tfidf_with_keywords=train_tfidf.fit_transform(sentences)
X_tfidf_with_keywords=X_tfidf_with_keywords.toarray()


In [ ]:
X_tfidf_with_keywords.shape

In [ ]:
for i,h in enumerate(sentence_handler):
    for nc in h.noun_chunks:
        if company_symbol[i] in nc.text \
        or company_name[1] in nc.text \
        or company_processed_name[2] in nc.text:
            if nc.root.head.text in bag_of_keywords:
                
                j = np.where(bag_of_keywords==nc.root.head.text)[0][0]
                
                if nc.root.dep_ == spacy.symbols.nsubj:
                    
                    X_tfidf_with_keywords[i,j]=X_tfidf_with_keywords[i,j]*Polarity_Score[j]
                    
                if nc.root.dep_ == spacy.symbols.nsubjpass:
                    
                    X_tfidf_with_keywords[i,j]=X_tfidf_with_keywords[i,j]*(-Polarity_Score[j])
                    

In [ ]:

numOfrow = price_movement.shape[0]

Rate_Of_Change_Array =np.zeros((numOfrow,2))

for i in range(len(roc_future_past1[:])):
    
    Rate_Of_Change_Array[i]=[roc_past1_past2[i],roc_past2_past3[i]]

In [ ]:
X_train_data_final = np.concatenate((X_tfidf_with_keywords,Rate_Of_Change_Array),axis=1)
X_train_data_final.shape

In [ ]:
Y_stock_price_movement = np.array(price_movement[:],dtype='int32')
Y_stock_price_movement[Y_stock_price_movement == -1] = 0
Y_stock_price_movement = to_categorical(Y_stock_price_movement,num_classes=2)


In [ ]:
import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.utils import to_categorical

In [ ]:
print("Number of days when stock price increases: ", np.sum(Y_stock_price_movement[:,1] == 1) )
print("Number of days when stock price decreases: ", np.sum(Y_stock_price_movement[:,0] == 1) )

In [ ]:
DNN_model=Sequential()
input_dimension = X_train_data_final.shape[1]
DNN_model.add(Dense(512, activation='relu',input_dim = input_dimension))
DNN_model.add(Dropout(0.5))
DNN_model.add(Dense(512,activation='relu'))
DNN_model.add(Dropout(0.5))
DNN_model.add(Dense(512,activation='relu'))

DNN_model.add(Dense(2,activation='softmax'))
DNN_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
DNN_model.fit(X_train_data_final, Y_stock_price_movement,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_split=0.2)

# Test

In [ ]:
processed_news_test = np.array(test_news)
sentences_test = processed_news_test[:,3]
tokens_test = processed_news_test[:,4]
sentence_handler_test = processed_news_test[:,5]
bigrams_test = processed_news_test[:,6]
company_symbol_test = processed_news_test[:,7]
company_name_test = processed_news_test[:,8]
company_processed_name_test = processed_news_test[:,9]
roc_future_past1_test = 100*(processed_news_test[:,10] - processed_news_test[:,11])/processed_news_test[:,11]
roc_past1_past2_test = 100*(processed_news_test[:,11] - processed_news_test[:,12])/processed_news_test[:,12]
roc_past2_past3_test = 100*(processed_news_test[:,12] - processed_news_test[:,13])/processed_news_test[:,13]
price_movement_test = processed_news_test[:,14]


In [ ]:
full_tfidf_test = TfidfVectorizer(lowercase=False,min_df=1,vocabulary=bag_of_keywords,use_idf=False)
X_full_tfidf_test = full_tfidf_test.fit_transform(sentences_test)
X_full_tfidf_test = X_full_tfidf_test.toarray()

In [ ]:
for i,h in enumerate(sentence_handler_test):
    for nouns in h.noun_chunks:
        if company_symbol_test[i] in nouns.text \
        or company_name_test[1] in nouns.text \
        or company_processed_name_test[2] in nouns.text:
            if nouns.root.head.text in bag_of_keywords:
                
                j=np.where(bag_of_keywords==nouns.root.head.text)[0][0]
                
                if nouns.root.dep_ == spacy.symbols.nsubj:
                    X_full_tfidf_test[i,j]=X_full_tfidf_test[i,j]*Polarity_Score[j]
                    
                if nouns.root.dep_ == spacy.symbols.nsubjpass:
                    X_full_tfidf_test[i,j]=X_full_tfidf_test[i,j]*(-Polarity_Score[j])
                    

In [ ]:

numOfrow_test = price_movement_test.shape[0]

Rate_Of_Change_Array_Test =np.zeros((numOfrow_test,2))

for i in range(len(roc_future_past1_test[:])):
    
    Rate_Of_Change_Array_Test[i]=[roc_past1_past2_test[i],roc_past2_past3_test[i]]

In [ ]:
X_test_data_final = np.concatenate((X_full_tfidf_test,Rate_Of_Change_Array_Test),axis=1)
X_test_data_final.shape

In [ ]:
Y_true_stock_price_movement = np.array(price_movement_test[:],dtype = 'int32')
Y_true_stock_price_movement[Y_true_stock_price_movement == -1] = 0
Y_true_stock_price_movement = Y_true_stock_price_movement.tolist()


In [ ]:
predictions = DNN_model.predict_classes(X_test_data_final)

In [ ]:
from sklearn.metrics import accuracy_score
test_accuracy = accuracy_score(Y_true_stock_price_movement,predictions)

In [ ]:
test_accuracy

# Bigram Model

In [ ]:
tfidf_vectorizer_bigram = TfidfVectorizer(lowercase=False,min_df=1,use_idf=True,vocabulary=bag_of_bigram_keywords)


In [ ]:
news_tfidf_bigram = tfidf_vectorizer_bigram.fit_transform(sentences)
news_tfidf_bigram = news_tfidf_bigram.toarray()

In [ ]:
bigram_frequency = np.sum(news_tfidf_bigram,axis=0)

In [ ]:
positive_bigram_frequency = np.array(np.sum(news_tfidf_bigram[price_movement[:]==1.0],axis=0)*news_tfidf_bigram.shape[0]).reshape(news_tfidf_bigram.shape[1],)
positive_bigram_frequency[positive_bigram_frequency == 0] = 1
bigram_frequency_positive = np.sum(price_movement[:]==1.0)
PMI_positive_bigram = np.log(positive_bigram_frequency*sentences.shape[0]/\
                      bigram_frequency*bigram_frequency_positive)

In [ ]:
negative_bigram_frequency = np.array(np.sum(news_tfidf_bigram[price_movement[:]==-1.0],axis=0)*news_tfidf_bigram.shape[0]).reshape(news_tfidf_bigram.shape[1],)
negative_bigram_frequency[negative_bigram_frequency==0] = 1
bigram_frequency_negative = np.sum(price_movement[:]==-1.0)
PMI_negative_bigram = np.log(negative_bigram_frequency*sentences.shape[0]/\
                             bigram_frequency*bigram_frequency_negative)


In [ ]:
Polarity_Score_bigram = PMI_positive_bigram - PMI_negative_bigram
Polarity_Score_bigram.shape

In [ ]:
train_tfidf_bigram = TfidfVectorizer(lowercase=False,min_df=1,vocabulary=bag_of_bigram_keywords,use_idf=False)
X_tfidf_with_bigram_keywords = train_tfidf_bigram.fit_transform(sentences)
X_tfidf_with_bigram_keywords = X_tfidf_with_bigram_keywords.toarray()


In [ ]:
X_tfidf_with_bigram_keywords.shape

In [ ]:
for i,h in enumerate(sentence_handler):
    for nouns in h.noun_chunks:
        if company_symbol[i] in nouns.text \
        or company_name[1] in nouns.text \
        or company_processed_name[2] in nouns.text:
            if nouns.root.head.text in bag_of_bigram_keywords:
                
                j = np.where(bag_of_bigram_keywords==nouns.root.head.text)[0][0]
                
                if nouns.root.dep_ == spacy.symbols.nsubj:
                    X_tfidf_with_bigram_keywords[i,j]=X_tfidf_with_bigram_keywords[i,j]*Polarity_Score_bigram[j]
                    
                if nouns.root.dep_ == spacy.symbols.nsubjpass:
                    X_tfidf_with_bigram_keywords[i,j]=X_tfidf_with_bigram_keywords[i,j]*(-Polarity_Score_bigram[j])
                    

In [ ]:
X_bigram_train_data_final = np.concatenate((X_tfidf_with_bigram_keywords,Rate_Of_Change_Array),axis=1)
X_bigram_train_data_final.shape

In [ ]:
DNN_bigram_model=Sequential()
input_dimension = X_bigram_train_data_final.shape[1]
DNN_bigram_model.add(Dense(512, activation='relu',input_dim = input_dimension))
DNN_bigram_model.add(Dropout(0.5))
DNN_bigram_model.add(Dense(512,activation='relu'))
DNN_bigram_model.add(Dropout(0.5))
DNN_bigram_model.add(Dense(512,activation='relu'))

DNN_bigram_model.add(Dense(2,activation='softmax'))
DNN_bigram_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
DNN_bigram_model.fit(X_bigram_train_data_final, Y_stock_price_movement,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_split=0.2)

# Model With Features: Unigram, Bigram and Event Tag

In [ ]:
category_tags = set(['published','investment','bankrupt','government','acquisition','suit'])

category_size=100

for i in range(10):
    new_words=[]

    for tag in category_tags:
        
        if tag in model.wv.vocab.keys():
            new_words.extend(model.most_similar(tag))
            
    for word in new_words:
        if (word[0].isalpha()) \
        and (word[0].islower()) \
        and (len(word[0])>3):
            #print(word[0])
            category_tags.add(word[0])
            if len(category_tags) == category_size:
                break
    if len(category_tags) == category_size:
        break

In [ ]:
category_tags = np.array(list(category_tags))
category_tags.shape

In [ ]:
category_vectorizer = CountVectorizer(lowercase=False,min_df=1,vocabulary=category_tags)
X_category_vectorizer= category_vectorizer.fit_transform(sentences)
X_category_vectorizer=X_category_vectorizer.toarray()

In [ ]:
category_tfidf=TfidfVectorizer(lowercase=False,min_df=1,vocabulary=category_tags)
X_category_tfidf=category_tfidf.fit_transform(sentences)
X_category_tfidf=X_category_tfidf.toarray()

In [ ]:
positive_category_frequency=np.array(np.sum(X_category_tfidf[price_movement[:]==1.0],axis=0)*X_category_tfidf.shape[0]).reshape(X_category_tfidf.shape[1],)
positive_category_frequency[positive_category_frequency==0]=1
category_frequency=np.sum(X_category_tfidf,axis=0)
frequency_cat_positive=np.sum(price_movement[:]==1.0)
category_PMI_pos=np.log(positive_category_frequency*sentences.shape[0]/category_frequency*frequency_cat_positive)

In [ ]:
negative_category_frequency=np.array(np.sum(X_category_tfidf[price_movement[:]==-1.0],axis=0)*X_category_tfidf.shape[0]).reshape(X_category_tfidf.shape[1],)
negative_category_frequency[negative_category_frequency==0]=1
category_frequency=np.sum(X_category_tfidf,axis=0)
frequency_cat_negative=np.sum(price_movement[:]==-1.0)
category_PMI_neg=np.log(negative_category_frequency*sentences.shape[0]/category_frequency*frequency_cat_negative)

In [ ]:
polarity_score_category = category_PMI_pos - category_PMI_neg
polarity_score_category.shape

In [ ]:
full_feature = np.concatenate((bag_of_keywords,bag_of_bigram_keywords,category_tags))
full_feature,full_index=np.unique(full_feature,return_index=True)
polarity_score_all =np.concatenate((Polarity_Score,Polarity_Score_bigram,polarity_score_category))[full_index]


In [ ]:
all_tfidf = TfidfVectorizer(lowercase=False,min_df=1,vocabulary=full_feature,use_idf=False)
X_all_tfidf=all_tfidf.fit_transform(sentences)
X_all_tfidf=X_all_tfidf.toarray()

In [ ]:
for i,h in enumerate(sentence_handler):
    for nc in h.noun_chunks:
        if company_symbol[i] in nc.text or company_name[1] in nc.text or company_processed_name[2] in nc.text:
            if nc.root.head.text in full_feature:
                
                j = np.where(full_feature==nc.root.head.text)[0][0]
                
                if nc.root.dep_ == spacy.symbols.nsubj:
                    X_all_tfidf[i,j]=X_all_tfidf[i,j]*polarity_score_all[j]
                    
                if nc.root.dep_ == spacy.symbols.nsubjpass:
                    X_all_tfidf[i,j]=X_all_tfidf[i,j]*(-polarity_score_all[j])
                    

In [ ]:
X_all_tfidf.shape

In [ ]:
X_train_all_features =np.concatenate((X_all_tfidf,Rate_Of_Change_Array),axis=1)
X_train_all_features.shape

In [ ]:
DNN_model_all_features =Sequential()
input_dimension = X_train_all_features.shape[1]
DNN_model_all_features.add(Dense(480, activation='relu',input_dim = input_dimension))
DNN_model_all_features.add(Dropout(0.5))
DNN_model_all_features.add(Dense(480,activation='relu'))
DNN_model_all_features.add(Dropout(0.5))
DNN_model_all_features.add(Dense(480,activation='relu'))

DNN_model_all_features.add(Dense(2,activation='softmax'))
DNN_model_all_features.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
DNN_model_all_features.fit(X_train_all_features, Y_stock_price_movement,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_split=0.2)

# Test (Unigram+Bigram+Event Tag)

In [ ]:
full_tfidf_test_all = TfidfVectorizer(lowercase=False,min_df=1,vocabulary=full_feature,use_idf=False)
X_full_tfidf_test_all = full_tfidf_test_all.fit_transform(sentences_test)
X_full_tfidf_test_all = X_full_tfidf_test_all.toarray()

In [ ]:
for i,h in enumerate(sentence_handler_test):
    for nouns in h.noun_chunks:
        if company_symbol_test[i] in nouns.text \
        or company_name_test[1] in nouns.text \
        or company_processed_name_test[2] in nouns.text:
            
            if nouns.root.head.text in full_feature:
                
                j=np.where(full_feature==nouns.root.head.text)[0][0]
                
                if nouns.root.dep_ == spacy.symbols.nsubj:
                    X_full_tfidf_test_all[i,j]=X_full_tfidf_test_all[i,j]*polarity_score_all[j]
                    
                if nouns.root.dep_ == spacy.symbols.nsubjpass:
                    X_full_tfidf_test_all[i,j]=X_full_tfidf_test_all[i,j]*(-polarity_score_all[j])
                    

In [ ]:
X_test_data_all_feature = np.concatenate((X_full_tfidf_test_all,Rate_Of_Change_Array_Test),axis=1)
X_test_data_all_feature.shape

In [ ]:
predictions_all_feature = DNN_model_all_features.predict_classes(X_test_data_all_feature)


In [ ]:
from sklearn.metrics import accuracy_score
test_accuracy_all_feature = accuracy_score(Y_true_stock_price_movement,predictions_all_feature.tolist())


In [ ]:
test_accuracy_all_feature